In [1]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

In [2]:
def load_images_from_folder(folders):
    class_images = []
    for folder in folders:
        images = []
        for filename in os.listdir(folder):
            img = Image.open(os.path.join(folder,filename))
            if img is not None:
                images.append(np.array(img))
        class_images.append(images)
    
    return class_images.copy()

In [3]:
def defined_input(class_images):
    features = []
    for images in class_images:
        for img in images:
            features.append(img)
    features = np.array(features)
    features = tf.keras.applications.inception_v3.preprocess_input(features, data_format=None)
    return features

In [4]:
#เตรียม output
def defined_output(class_images):
    label = []
    for count, images in enumerate(class_images):
        for i in range(len(images)):
            classes = len(class_images)
            output = np.zeros((classes,), dtype=int)
            output[count] = 1
            label.append(output)
    return np.array(label)

In [5]:
#train
foldersname = []
for i in range(14):
    foldersname.append('./images/Tail/train/class' + str(i)) #path_to_folder, change file path this.

class_images = load_images_from_folder(foldersname)

X_train = np.copy(defined_input(class_images))
y_train = np.copy(defined_output(class_images))

In [7]:
#validation
foldersname = []
for i in range(14):
    foldersname.append('./images/Tail/validation/class' + str(i)) #path_to_folder, change file path this.

class_images = load_images_from_folder(foldersname)

X_validation = np.copy(defined_input(class_images))
y_validation = np.copy(defined_output(class_images))

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout , GlobalAveragePooling2D

In [9]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3))

base_model.trainable = False #Freeze Parameters

inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(14, activation='softmax')(x)

In [10]:
model = Model(inputs, predictions)

In [11]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)              

In [ ]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = (X_validation, y_validation))

# Fine Tuning

In [ ]:
# base_model.trainable = True
# fine_tune_at = 249

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable =  False

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate / 10),
#               loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
#               metrics=['accuracy'])

In [ ]:
# model.fit(X_train, y_train, batch_size = 32, epochs = 10, initial_epoch = history.epoch[-1], validation_data = (X_validation, y_validation))

In [ ]:
#save model
model.save('./model_latest/tail/tail_v2.h5') #path_to_save_model, change file path this.